In [1]:
import os
import codecs
import cv2
import numpy as np
from keras import backend as K
from keras.layers import Input, Dense, Activation, Conv2D, Reshape
from keras.layers import BatchNormalization, Lambda, MaxPooling2D, Dropout
from keras.layers.merge import add, concatenate
from keras.callbacks import EarlyStopping,Callback
from keras.layers.recurrent import GRU
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
CHARS = ['京', '沪', '津', '渝', '冀', '晋', '蒙', '辽', '吉', '黑',
         '苏', '浙', '皖', '闽', '赣', '鲁', '豫', '鄂', '湘', '粤',
         '桂', '琼', '川', '贵', '云', '藏', '陕', '甘', '青', '宁',
         '新',
         '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
         'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K',
         'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z'
         ]
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
NUM_CHARS = len(CHARS)
#必要参数
num_channels = 3
ti = './car_pic/image/train/' #训练图片目录
tl = './car_pic/image/train_labels.txt' #训练标签文件
vi = './car_pic/image/val/'  #验证图片目录
vl = './car_pic/image/val_labels.txt' #验证标签文件
img_size = [128,40] #训练图片宽和高
label_len = 7 #标签长度
dir_log = './logs/'
c = './car_pic/image/' #checkpoints format string
num_epochs = 200 #number of epochs
start_of_epoch = 0

#网络参数
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 512
batch_size = 1

Using TensorFlow backend.


In [2]:
def ImageDataGenerator(img_dir, label_file, batch_size, img_size, input_length, num_channels, label_len):
    num_examples = 0
    next_index = 0
    filenames = []
    labels = []
    with codecs.open(label_file,mode='r', encoding='utf-8') as f:
        for line in f:
            filename, label = parse_line(line)
            filenames.append(filename)
            labels.append(label)
            num_examples += 1
    labels = np.array(labels) #没有必要浮点话？

    while True:
        num_epoches = 0
        # 洗乱数据
        if next_index == 0:
            perm = np.arange(num_examples)
            np.random.shuffle(perm)
            filenames = [filenames[i] for i in perm]
            labels_shuttle = labels[perm]
        #  
        start = next_index
        end = next_index + batch_size
        if end >= num_examples:
            next_index = 0
            num_epoches += 1
            end = num_examples
            batch_size = num_examples - start
        else:
            next_index = end
        images = np.zeros([batch_size, img_size[1], img_size[0], num_channels])
        # labels = np.zeros([batch_size, label_len])
        for j, i in enumerate(range(start, end)):
            fname = filenames[i]
            img = cv2.imdecode(np.fromfile(img_dir+fname.strip()+'.jpg', dtype=np.uint8), 1)
            # cv2.imshow('test',img)
            images[j, ...] = img
        images = np.transpose(images, axes=[0, 2, 1, 3])
        labels = labels_shuttle[start:end, ...]
        input_length = np.zeros([batch_size, 1])
        label_length = np.zeros([batch_size, 1])
        input_length[:] = input_length
        label_length[:] = label_len
        outputs = {'ctc': np.zeros([batch_size])}
        inputs = {'the_input': images,
                  'the_labels': labels,
                  'input_length': input_length,
                  'label_length': label_length,
                  }
        yield inputs, outputs

#产生数据
train_gen = ImageDataGenerator(img_dir=ti,
                                 label_file=tl,
                                 batch_size=batch_size,
                                 img_size=img_size,
                                 input_length=pred_length,
                                 num_channels=num_channels,
                                 label_len=label_len)

val_gen = ImageDataGenerator(img_dir=vi,
                                 label_file=vl,
                                 batch_size=batch_size,
                                 img_size=img_size,
                                 input_length=pred_length,
                                 num_channels=num_channels,
                                 label_len=label_len)


NameError: name 'pred_length' is not defined